<a href="https://colab.research.google.com/github/Dibya069/Generative_AI-Codes-to-learn-/blob/main/RAG_%2B_PineCone_%2B_Memory_management_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain pinecone-client openai langchain-community tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/4

In [ ]:
import os
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as lang_pine
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel

In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
conversation_history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
  ]

In [ ]:
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    # Load openai model embeddings and pinecone
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

    # Create Pinecone index if it doesn't exist
    if index_name not in pc.list_indexes():
        pc.create_index(index_name, dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region='us-east-1'))


    # Encode and upsert messages into Pinecone index
    index = pc.Index(index_name)
    for i, message in enumerate(history):
        embedding = embeddings.embed_query(message)
        message_id = f"msg-{i}"
        index.upsert([(message_id, embedding)])

    print(index.describe_index_stats())

add_embeddings_to_pinecone(conversation_history)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19}},
 'total_vector_count': 19}


In [ ]:
def retrieve_relevant_history(query, history, index_name='chat-history'):
    # Load OpenAI model embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

    # Encode the query to get its embedding
    query_embedding = embeddings.embed_query(query)

    # Query Pinecone for relevant history
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    # Retrieve top 3 relevant messages
    result = index.query(vector = query_embedding, top_k = 3, include_metadata = True)

    print(result)
    # Extract relevant message IDs and their associated metadata
    relevant_messages = []
    for match in result['matches']:
        message_id = match['id']
        # Fetch the relevant message by its ID
        relevant_messages.append(conversation_history[int(message_id.split('-')[1])])  # Ensure IDs are integers

    return relevant_messages


In [ ]:
def prepare_prompt(final_test_prompt, history):
  # Retrieve relevant history (message 7, 8, and 10) using Pinecone
  relevant_messages = retrieve_relevant_history(final_test_prompt, history)

  # Combine relevant history and final test prompt
  context_referred = "\n".join(relevant_messages)
  prepared_prompt = f"{context_referred}\nUser: {final_test_prompt}\nBot:"

  return prepared_prompt, context_referred

def generate_openai_response(prepared_prompt):
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
  response = llm.predict(prepared_prompt)

  return response

In [ ]:
# Call prepare_prompt
final_test_prompt = "Do you think it will help me stay fit?"
prepared_prompt, context_referred = prepare_prompt(final_test_prompt, conversation_history)


# Generate the response
response = generate_openai_response(prepared_prompt)

{'matches': [{'id': 'msg-6', 'score': 0.791899502, 'values': []},
             {'id': 'msg-7', 'score': 0.762535572, 'values': []},
             {'id': 'msg-8', 'score': 0.762237608, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [ ]:
# Print the results
print(f"Final Test Prompt: {final_test_prompt} \n")
print(f"Context Referred: \n{context_referred} \n")
print(f"Final Test Prompt Response: {response} \n")

Final Test Prompt: Do you think it will help me stay fit? 

Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Final Test Prompt Response: Yes, 'Atomic Habits' provides valuable insights on how to build good habits and break bad ones, which can be very helpful in maintaining a regular exercise routine and staying fit. 



In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Goodbye!")
        break

    prepared_prompt, context_referred = prepare_prompt(user_input, conversation_history)
    response = generate_openai_response(prepared_prompt)

    print(f"Context Referred: \n{context_referred} \n")
    print(f"Response: {response} \n")
    print("\n ------------------------------------------ \n")

User: Do you think it will help me stay fit? 
{'matches': [{'id': 'msg-6', 'score': 0.803957105, 'values': []},
             {'id': 'msg-7', 'score': 0.775736928, 'values': []},
             {'id': 'msg-8', 'score': 0.774762511, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Response: Yes, 'Atomic Habits' provides valuable insights into forming good habits, which can definitely help in maintaining a regular exercise routine and staying fit. 


 ------------------------------------------ 

User: why did you give me this type of movie recommendations ??
{'matches': [{'id': 'msg-13',

***

## Trying to add the recent conversations to the history and Vectordb also, for future conversations reference

In [ ]:
def add_new_conversation_embedding_to_pinecone(message, index_name='chat-history'):
    # Load OpenAI model embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

    # Create Pinecone client and index
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    # Get the embedding for the message
    embedding = embeddings.embed_query(message)

    # Generate a unique message ID based on the history length
    message_id = f"msg-{len(conversation_history)}"

    # Upsert the message embedding into Pinecone
    index.upsert([(message_id, embedding)])

    print(f"Added to Pinecone: {message_id}, {message}")

In [ ]:
def generate_openai_response(prepared_prompt, user_query):
    llm = ChatOpenAI(model_name='gpt-4', temperature=0)
    response = llm.predict(prepared_prompt)  # Pass the prepared prompt

    # Append user query and AI response to history
    conversation_history.append(f"user: {user_query} | bot: {response}")  # Append user query

    # Add both the user query and the generated response to Pinecone
    add_new_conversation_embedding_to_pinecone(f"user: {user_query} | bot: {response}")

    return response

In [ ]:
# Example of calling the functions
final_test_prompt = "Do you think it will help me stay fit?"

# Prepare the prompt
prepared_prompt, context_referred = prepare_prompt(final_test_prompt, conversation_history)

# Generate the response and update the conversation history
response = generate_openai_response(prepared_prompt, final_test_prompt)


{'matches': [{'id': 'msg-6', 'score': 0.791899502, 'values': []},
             {'id': 'msg-7', 'score': 0.762535572, 'values': []},
             {'id': 'msg-8', 'score': 0.762237608, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
Added to Pinecone: msg-21, user: Do you think it will help me stay fit? | bot: Yes, 'Atomic Habits' provides valuable insights on how to build good habits and break bad ones, which can be very helpful in maintaining a regular exercise routine and staying fit.


In [ ]:
conversation_history

["1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
 "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
 '3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?',
 '4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.',
 "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
 '6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?',
 '7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.',
 '8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.',
 "9: User: I'll start with ru

***
***

### Cell 1 [Required Libreries Installation]

In [ ]:
!pip install langchain pinecone-client openai langchain-community tiktoken sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

## Cell 2: Import all the required Libreries

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as lang_pine
from langchain.chat_models import ChatOpenAI
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2Tokenizer

## Cell 3: Implementing all the API Keys

In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

## Cell 4: Define the conversation history

In [ ]:
conversation_history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
  ]

## Cell 5: Define the function to add embeddings to Pinecone

In [ ]:
def add_embeddings_to_pinecone(history, index_name='chat-history'):
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

    # Create Pinecone index if it doesn't exist
    if index_name not in pc.list_indexes():
        pc.create_index(index_name, dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region='us-east-1'))


    # Encode and upsert messages into Pinecone index
    index = pc.Index(index_name)
    for i, message in enumerate(history):
        embedding = embeddings.embed_query(message)
        message_id = f"msg-{i}"
        index.upsert([(message_id, embedding)])

    print(index.describe_index_stats())

add_embeddings_to_pinecone(conversation_history)

<ipython-input-5-358e59932aae>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19}},
 'total_vector_count': 19}


## Cell 6: Define the RAG mechanism [with transformer model.]

In [ ]:
# Function to upscale the embedding to 1536 dimensions
def upscale_embedding(embedding, target_dim=1536):
    if len(embedding) < target_dim:
        repeat_times = target_dim // len(embedding)  # Number of times to repeat
        upscale_embedding = np.tile(embedding, repeat_times)
        return upscale_embedding[:target_dim]  # Ensure it's exactly the target dimension
    return embedding

# Function to retrieve relevant history using transformer embeddings
def retrieve_relevant_history_1(query, history, index_name='chat-history'):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')  # Embeddings are of size 384
    query_embedding = model.encode(query)

    query_embedding_1536 = upscale_embedding(query_embedding, target_dim=1536)

    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    result = index.query(vector=query_embedding_1536.tolist(), top_k=3, include_metadata=True)

    relevant_messages = []
    for match in result['matches']:
        message_id = match['id']
        relevant_messages.append(history[int(message_id.split('-')[1])])  # Ensure IDs are integers

    return relevant_messages

## Cell 7: Define the RAG mechanism [with OpenAI Embedding model.]

In [ ]:
def retrieve_relevant_history_2(query, history, index_name='chat-history'):
    embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))
    query_embedding = embeddings.embed_query(query)

    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name)

    result = index.query(vector = query_embedding, top_k = 3, include_metadata = True)

    relevant_messages = []
    for match in result['matches']:
        message_id = match['id']
        relevant_messages.append(conversation_history[int(message_id.split('-')[1])])

    return relevant_messages


- To retrive the information according to the User Query, I used 2 method
  1. Transformers method [`sentence-transformers/all-MiniLM-L12-v2`]
  2. OpneAI method [`OpenAIEmbedding from Langchain`]

## Cell 8: Function to prepare the prompt [For Transformers and For OpenAI]



In [ ]:
## Prompt Preparation
class PromptPreparation:
  def __init__(self, history):
    self.history = history

  ## For Transformers
  def prepare_prompt_1(self, final_test_prompt):
    relevant_messages = retrieve_relevant_history_1(final_test_prompt, self.history)

    context_referred = "\n".join(relevant_messages)
    prepared_prompt = f"{context_referred}\nUser: {final_test_prompt}\nBot:"

    return prepared_prompt, context_referred

  ## For OpenAI
  def prepare_prompt_2(self, final_test_prompt):
    relevant_messages = retrieve_relevant_history_2(final_test_prompt, self.history)

    context_referred = "\n".join(relevant_messages)
    prepared_prompt = f"{context_referred}\nUser: {final_test_prompt}\nBot:"

    return prepared_prompt, context_referred


# For Generating Response
def generate_openai_response(prepared_prompt):
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
  response = llm.predict(prepared_prompt)

  return response

## Cell 9: Function to test the prompt [For Transformers and For OpenAI]

In [ ]:
## ## For Transformers Response
def test_prompt_1(user_input):
  Transformer_PromptPreparation = PromptPreparation(conversation_history)
  prepared_prompt, context_referred = Transformer_PromptPreparation.prepare_prompt_1(user_input)
  response = generate_openai_response(prepared_prompt)

  print(f"Context Referred: \n{context_referred} \n")
  print(f"Response: {response} \n")
  print("\n ------------------------------------------ \n")



## For OpenAI Retriver
def test_prompt_2(user_input):
  OpenAI_PromptPreparation = PromptPreparation(conversation_history)
  prepared_prompt, context_referred = OpenAI_PromptPreparation.prepare_prompt_2(user_input)
  response = generate_openai_response(prepared_prompt)

  print(f"Context Referred: \n{context_referred} \n")
  print(f"Response: {response} \n")
  print("\n ------------------------------------------ \n")

In [ ]:
final_test_prompt = "Do you think it will help me stay fit?"


print(test_prompt_1(final_test_prompt)) ## Retrive information using Transformer model
print(test_prompt_2(final_test_prompt)) ## Retrive information using OpneAI Embbedings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-8-2d24a8bda959>:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4', temperature=0)
<ipython-input-8-2d24a8bda959>:28: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prepared_prompt)


Context Referred: 
16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.
17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.
20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences. 

Response: Absolutely, traveling often involves a lot of walking and physical activities which can definitely help you stay fit. 


 ------------------------------------------ 

None
Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Response: Yes, 'Atomic Habits' provides valuable insig

## Cell 10: Ensuring, do not exceed 225 tokens.

In [ ]:
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # GPT-3/GPT-4 compatible tokenizer


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def count_tokens(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

def prepare_final_prompt_and_truncate(context, final_test_prompt, response, token_limit=225):
    if isinstance(context, str):
        context = context.split('\n')

    context_str = "\n".join(context)
    final_prompt = f"User: {final_test_prompt}\nBot: {response}"
    combined_text = f"{context_str}\n{final_prompt}"

    total_tokens = count_tokens(combined_text)

    print("Token Limit : ", total_tokens)
    print("\n")

    if total_tokens > token_limit:
        while total_tokens > token_limit and context:
            context.pop(0)
            context_str = "\n".join(context)
            combined_text = f"{context_str}\n{final_prompt}"
            total_tokens = count_tokens(combined_text)

    return combined_text

In [ ]:
## For Transformers Response
def test_prompt_1(user_input):
  Transformer_PromptPreparation = PromptPreparation(conversation_history)
  prepared_prompt, context_referred = Transformer_PromptPreparation.prepare_prompt_1(user_input)
  response = generate_openai_response(prepared_prompt)

  return context_referred, response


final_test_prompt = "Do you think it will help me stay fit?"
context_referred, response =  test_prompt_1(final_test_prompt)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
final_text = prepare_final_prompt_and_truncate(context_referred, final_test_prompt, response, token_limit=225)
print(final_text)

Token Limit :  136


16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.
17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.
10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.
User: Do you think it will help me stay fit?
Bot: Absolutely, especially hiking. It's a great way to stay active and enjoy nature. Learning a new instrument can also help with mental fitness.


***

### Try to convert this into chatbot conversation

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Goodbye!")
        break

    OpenAI_PromptPreparation = PromptPreparation(conversation_history)
    prepared_prompt, context_referred = OpenAI_PromptPreparation.prepare_prompt_2(user_input)
    response = generate_openai_response(prepared_prompt)

    print(f"\nContext Referred: \n{context_referred} \n")
    print(f"Response: {response}")
    print("\n ------------------------------------------ \n")

User: Do you think it will help me stay fit?

Context Referred: 
7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.
8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.
9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book. 

Response: Yes, 'Atomic Habits' provides valuable insights on how to build good habits and break bad ones, which can be very helpful in maintaining a regular exercise routine and staying fit.

 ------------------------------------------ 

User: Why did give me this type of movie recommendations ??

Context Referred: 
14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.
15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.
16: User: Great c